In [1]:
import pandas as pd
import numpy as np
from functions import *
import re

%load_ext autoreload
%autoreload 2

pd.set_option('future.no_silent_downcasting', True)

In [2]:
# set paths to data
path = '/Users/johan/Library/CloudStorage/GoogleDrive-johan.oelgaard@gmail.com/My Drive/04 Økonomi/10 Thesis/Data'

# read daily market data from eikon
daily = 'eikon_daily.xlsx'
eikon_dfs = pd.read_excel(path + '/' + daily, sheet_name=None)
eikon_keys = eikon_dfs.keys()

In [3]:
# load trade data
trade_values_df = eikon_dfs['Trade Values'].iloc[:,1:]
# set up multi-index for the columns
trade_values_df.columns = pd.MultiIndex.from_arrays(trade_values_df.iloc[:2].values)
# drop the first two rows as they are now headers
trade_values_df = trade_values_df.iloc[2:].reset_index(drop=True)
# set the first column as index
trade_values_df.set_index(trade_values_df.columns[0], inplace=True)
trade_values_df.index.name = "timestamp"
trade_values_df = trade_values_df.sort_index(axis=1, level=0)
# keep only trade close values
trade_values_df = trade_values_df.loc[:, (slice(None), 'Trade Close')]
# set 0 values to NaN
trade_values_df = trade_values_df.replace(0, np.nan)

# backward fill the data for each ticker
idx = pd.IndexSlice

# loop over the tickers that are actually in the df
for ticker in trade_values_df.columns.get_level_values(0).unique():
    # extract the sub-dataframe for this ticker using .loc with IndexSlice
    subdf = trade_values_df.loc[:, idx[ticker, :]]
    
    # find the index range where the ticker has any valid data
    valid_idx = subdf.dropna(how='all').index

    # use backward fill in the date range
    trade_values_df.loc[valid_idx.max():valid_idx.min(), idx[ticker, :]] = trade_values_df.loc[valid_idx.max():valid_idx.min(), idx[ticker, :]].bfill()

# stack first level of columns to rows
trade_values_df = trade_values_df.stack(level=0).reset_index()
# rename columns
trade_values_df.columns = ['timestamp', 'Ticker', 'Trade Close']
# set first column as index
trade_values_df.set_index('timestamp', inplace=True)

# calculate the daily returns
trade_values_df = trade_values_df.sort_values(by=['Ticker', 'timestamp'], ascending=[True, True])
trade_values_df['stkre'] = trade_values_df.groupby('Ticker')['Trade Close'].pct_change()


/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)
/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_6322/2510852289.py:31: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  trade_values_df = trade_values_df.stack(level=0).reset_index()


In [4]:
# load index data
omxcpi = eikon_dfs['OMXCPI'].iloc[:,1:]
# set first row as header
omxcpi.columns = omxcpi.iloc[0]
# drop the first row as it is now header
omxcpi = omxcpi.iloc[1:].reset_index(drop=True)
# set the first column as index
omxcpi.set_index(omxcpi.columns[0], inplace=True)
omxcpi = omxcpi.sort_index(axis=1)
# keep only closing values
omxcpi = omxcpi.loc[:,'Trade Close']
# convert to dataframe
omxcpi = pd.DataFrame(omxcpi)
# rename columns
omxcpi.columns = ['OMXCPI']

omxcpi = omxcpi.sort_index(ascending=True)
omxcpi['mktre'] = omxcpi['OMXCPI'].pct_change()

/opt/anaconda3/lib/python3.11/site-packages/pandas/core/indexes/base.py:7588: FutureWarning: Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.
  return Index(sequences[0], name=names)


In [5]:
# calculate the rolling beta
# join the two dataframes on index
beta = trade_values_df.join(omxcpi, how='left')
# drop Trade Close and OMXCPI columns and calculate beta
beta = beta.drop(columns=['Trade Close', 'OMXCPI']).dropna()
beta= beta.groupby('Ticker').apply(rolling_beta)

# create df
beta = beta.reset_index()
beta.columns = ['ticker', 'timestamp', 'beta']
# set the index to timestamp
beta.set_index('timestamp', inplace=True)

# save the beta to csv
beta.to_csv('data/beta.csv')

/var/folders/yw/jsw5n53s1cb1s2q6tt0msrm00000gn/T/ipykernel_6322/3090447612.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  beta= beta.groupby('Ticker').apply(rolling_beta)
